In [1]:
import joblib
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
import seaborn as sns

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve, classification_report


Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
import os
os.environ['QT_QPA_PLATFORM']='offscreen'

In [ ]:
df = pd.read_csv(r"./data/bank_data.csv")
df.head()

In [7]:

def import_data(pth):
    '''
    returns dataframe for the csv found at pth
    
    input:
            pth: a path to the csv
    output:
            df: pandas dataframe
    '''	
    df = pd.read_csv(pth)
    return df

In [19]:


def perform_eda(df):
    '''
    Perform exploratory data analysis (EDA) on the dataframe and save figures to the images/eda directory.
    
    Input:
        df: pandas DataFrame - the dataframe on which EDA will be performed.
    
    Output:
        None - figures are saved to files.
    '''

    try:
        # Convert 'Attrition_Flag' into a binary 'Churn' column if needed
        if 'Churn' not in df.columns:
            df['Churn'] = df['Attrition_Flag'].apply(lambda val: 0 if val == "Existing Customer" else 1)

        # Path to the directory where the graphs will be saved
        save_path = './images/eda/'

        # Check if the directory exists; if not, create it.
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # Function to save histogram of 'Churn'
        def save_churn_hist():
            plt.figure(figsize=(20,10))
            df['Churn'].hist()
            file_path = os.path.join(save_path, 'histogram_churn.png')
            plt.savefig(file_path)
            plt.close()

        # Function to save histogram of 'Customer_Age'
        def save_customer_age_hist():
            plt.figure(figsize=(20,10))
            df['Customer_Age'].hist()
            file_path = os.path.join(save_path, 'customer_age_hist.png')
            plt.savefig(file_path)
            plt.close()

        # Function to save bar plot of 'Marital_Status'
        def save_marital_status_bar():
            plt.figure(figsize=(20,10))
            df['Marital_Status'].value_counts(normalize=True).plot(kind='bar')
            file_path = os.path.join(save_path, 'marital_status_bar.png')
            plt.savefig(file_path)
            plt.close()

        # Function to save histogram of 'Total_Trans_Ct' with KDE
        def save_transactions_hist():
            plt.figure(figsize=(20,10))
            sns.histplot(df['Total_Trans_Ct'], stat='density', kde=True)
            file_path = os.path.join(save_path, 'total_trans_hist.png')
            plt.savefig(file_path)
            plt.close()

        # Function to save heatmap of correlations
        def save_correlation_heatmap():
            plt.figure(figsize=(20,10))
            sns.heatmap(df.corr(), annot=False, cmap='Dark2_r', linewidths=2)
            file_path = os.path.join(save_path, 'correlation_heatmap.png')
            plt.savefig(file_path)
            plt.close()

        # Call functions to generate and save plots
        save_churn_hist()
        save_customer_age_hist()
        save_marital_status_bar()
        save_transactions_hist()
        save_correlation_heatmap()

    except Exception as e:
        print(f"An error occurred during EDA: {e}")



In [25]:
def encoder_helper(df, category_lst, response='Churn'):
    '''
    Helper function to turn each categorical column into a new column with
    the proportion of the response (e.g., churn) for each category.

    Input:
        df: pandas DataFrame - The dataframe to process.
        category_lst: list - A list of column names that contain categorical features.
        response: str - The name of the response column (default is 'Churn').

    Output:
        df: pandas DataFrame - The dataframe with new columns for each categorical variable encoded.
    '''

    def encode_column(column):
        '''
        Encodes a single categorical column using mean of the response variable.

        Input:
            column: str - The name of the column to encode.

        Output:
            None - The function directly modifies the dataframe in place.
        '''
        # Group by the category and calculate the mean of the response
        group_means = df.groupby(column).mean()[response]

        # Map the mean response to each entry in the dataframe
        encoded_column_name = f"{column}_{response}"
        df[encoded_column_name] = df[column].map(group_means)

    # Apply encoding to each category in the list
    for category in category_lst:
        encode_column(category)

    return df

# Example usage:
# Assuming 'df' is your DataFrame and it includes the 'Gender', 'Education_Level', 'Marital_Status',
# 'Income_Category', and 'Card_Category' columns that you want to encode.
category_list = ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']



In [39]:


def perform_feature_engineering(df, response):
    '''
    Perform feature engineering to prepare the dataset for machine learning models
    by selecting specified columns and splitting the data into training and testing sets.
    
    Input:
        df: pandas DataFrame - The dataframe to process.
        response: str - The name of the response column to be used as the target variable.
    
    Output:
        X_train: DataFrame - Training feature data.
        X_test: DataFrame - Testing feature data.
        y_train: Series - Training target data.
        y_test: Series - Testing target data.
        
    Raises:
        Exception: Captures and logs general exceptions if the feature engineering process fails.
    '''

    try:
        # Extracting the response variable
        y = df[response]
        
        # Define the columns to keep in the feature matrix X
        keep_cols = ['Customer_Age', 'Dependent_count', 'Months_on_book',
                     'Total_Relationship_Count', 'Months_Inactive_12_mon',
                     'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
                     'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
                     'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
                     'Gender_Churn', 'Education_Level_Churn', 'Marital_Status_Churn', 
                     'Income_Category_Churn', 'Card_Category_Churn']
        
        # Ensure all specified columns are in the DataFrame, to avoid KeyError
        filtered_cols = [col for col in keep_cols if col in df.columns]
        
        # Creating the feature matrix X with the specified columns
        X = df[filtered_cols]
        
        # Splitting the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        return X_train, X_test, y_train, y_test
    
    except KeyError as e:
        print(f"KeyError: One or more columns not found in the dataframe: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise



In [56]:
def train_models(X_train, X_test, y_train, y_test):
    '''
    Trains logistic regression and random forest models, evaluates them with classification reports,
    saves the models to disk, and returns training and testing predictions.

    Input:
        X_train: DataFrame - Training feature data.
        X_test: DataFrame - Testing feature data.
        y_train: Series - Training target data.
        y_test: Series - Testing target data.

    Output:
        Tuple - Contains y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf
    '''
    try:
        # Ensure the models directory exists
        models_dir = './models/'
        if not os.path.exists(models_dir):
            os.makedirs(models_dir)

        # Initialize the models
        lrc = LogisticRegression(solver='lbfgs', max_iter=3000)
        rfc = RandomForestClassifier(random_state=42)

        # Set up parameter grid for Random Forest
        param_grid = {
            'n_estimators': [100, 200],
            'max_features': ['auto', 'sqrt'],
            'max_depth': [4, 5],
            'criterion': ['gini', 'entropy']
        }

        # GridSearch for optimal parameters in Random Forest
        cv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
        cv_rfc.fit(X_train, y_train)

        # Fit Logistic Regression
        lrc.fit(X_train, y_train)

        # Predictions
        y_train_preds_rf = cv_rfc.best_estimator_.predict(X_train)
        y_test_preds_rf = cv_rfc.best_estimator_.predict(X_test)
        y_train_preds_lr = lrc.predict(X_train)
        y_test_preds_lr = lrc.predict(X_test)

        # Save models
        joblib.dump(cv_rfc.best_estimator_, os.path.join(models_dir, 'rfc_model.pkl'))
        joblib.dump(lrc, os.path.join(models_dir, 'logistic_model.pkl'))

        return (y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf)

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example of how you might call this function
# Assume X_train, X_test, y_train, y_test are defined from your data preparation steps
# results = train_models(X_train, X_test, y_train, y_test)


In [82]:
from sklearn.metrics import classification_report, roc_curve, auc, RocCurveDisplay


def classification_report_image(y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf):
    '''
    Produces classification report for training and testing results and stores report as image in images folder.
    
    Input:
        y_train: training response values
        y_test: test response values
        y_train_preds_lr: training predictions from logistic regression
        y_train_preds_rf: training predictions from random forest
        y_test_preds_lr: test predictions from logistic regression
        y_test_preds_rf: test predictions from random forest

    Output:
        None
    '''

    def save_classification_report(y_true, y_pred, model_name, dataset_type):
        '''
        Saves the classification report as an image.
        
        Input:
            y_true: actual target values
            y_pred: predictions from model
            model_name: name of the model (e.g., 'logistic_regression', 'random_forest')
            dataset_type: type of data (e.g., 'train', 'test')
        
        Output:
            None - saves an image file of the classification report
        '''
        report = classification_report(y_true, y_pred, output_dict=True)
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        ax.text(0, 0, f"{model_name.upper()} {dataset_type.upper()} REPORT:\n" + 
                classification_report(y_true, y_pred), fontdict={'fontsize': 14}, va='top')
        filepath = f'./images/results/{model_name}_{dataset_type}_report.png'
        plt.savefig(filepath)
        plt.close()

    def plot_roc_curves():
        '''
        Plots ROC curves for both models and saves as an image.
        '''
        try:
            plt.figure(figsize=(15, 8))
            ax = plt.gca()
            lrc = joblib.load('./models/logistic_model.pkl')
            rfc_best = joblib.load('./models/rfc_model.pkl')

            models = [
                ('Random Forest',rfc_best , y_test),
                ('Logistic Regression', lrc, y_test)
                
            ]

            for name, model, preds in models:
                fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
                roc_auc = auc(fpr, tpr)
                RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=name).plot(ax=ax)

            plt.savefig('./images/results/roc_curves.png')
            plt.close()

        except Exception as e:
            print(f"An error occurred while plotting ROC curves: {e}")

    # Call the function to save classification reports
    save_classification_report(y_train, y_train_preds_lr, 'logistic_regression', 'train')
    save_classification_report(y_test, y_test_preds_lr, 'logistic_regression', 'test')
    save_classification_report(y_train, y_train_preds_rf, 'random_forest', 'train')
    save_classification_report(y_test, y_test_preds_rf, 'random_forest', 'test')

    # Plot and save the ROC curves
    plot_roc_curves()

    # Assume model saving and loading functions are correctly defined elsewhere

classification_report_image(y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf)



In [83]:
def load_model(model_path):
    '''
    Safely loads a machine learning model from the specified path.
    
    Input:
        model_path: str - Path to the machine learning model file.
    
    Output:
        model - Loaded model if successful, None otherwise.
    '''
    try:
        model = joblib.load(model_path)
        return model
    except Exception as e:
        print(f"Failed to load model from {model_path}: {e}")
        return None


def feature_importance_plot(model, X_data, output_pth):
    '''
    Creates and stores both traditional feature importances and SHAP summary plots for tree-based models.
    
    Input:
        model: The trained tree-based model object (e.g., RandomForest, XGBoost).
        X_data: pandas DataFrame - The feature data used for the analysis.
        output_pth: str - The directory path where the feature importance plots will be saved.
    
    Output:
        None - The function saves feature importance plots in the specified output path.
    '''
    def ensure_dir(directory):
        ''' Ensure the directory exists. If not, create it. '''
        if not os.path.exists(directory):
            os.makedirs(directory)

    def plot_feature_importances():
        ''' Plot and save traditional feature importances. '''
        try:
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1]
            names = [X_data.columns[i] for i in indices]

            plt.figure(figsize=(20, 5))
            plt.title("Feature Importance")
            plt.ylabel('Importance')
            plt.bar(range(X_data.shape[1]), importances[indices])
            plt.xticks(range(X_data.shape[1]), names, rotation=90, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_pth, 'feature_importances.png'))
            plt.close()
        except Exception as e:
            print(f"Error generating feature importance plot: {e}")

    def plot_shap_summary():
        ''' Plot and save SHAP summary plot. '''
        try:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_data)
            shap.summary_plot(shap_values, X_data, plot_type="bar", show=False)
            plt.savefig(os.path.join(output_pth, 'shap_summary_plot.png'))
            plt.close()
        except Exception as e:
            print(f"Error generating SHAP summary plot: {e}")

    # Ensure the output directory exists
    ensure_dir(output_pth)

    # Generate and save the plots
    plot_feature_importances()
    plot_shap_summary()

# Example usage:
# Assuming `model` is your trained RandomForestClassifier or similar,
# and `X_test` is your DataFrame of test features:
# output_path = './images/results/'
# feature_importance_plot(model, X_test, output_path)

# Load the model using the helper function
rfc_best = load_model(model_path)

if rfc:
    feature_importance_plot(rfc_best, X_test, output_path)


In [74]:
pth = "./data/bank_data.csv"
output_path = './images/results/'
model_path = './models/rfc_model.pkl'

df = import_data(pth)
perform_eda(df)
df_encoded = encoder_helper(df, category_list)
X_train, X_test, y_train, y_test = perform_feature_engineering(df, 'Churn')
(y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf) = train_models(X_train, X_test, y_train, y_test)
classification_report_image(y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf)


# Load the model using the helper function
rfc = load_model(model_path)

if rfc:
    feature_importance_plot(rfc, X_test, output_path)



In [81]:
classification_report_image(y_train, y_test, y_train_preds_lr, y_train_preds_rf, y_test_preds_lr, y_test_preds_rf)
